In [1]:
import osmnx as ox
import pandas as pd
import numpy as np

In [ ]:
# longitude is x, latitude is y

G = ox.graph_from_place('San Francisco, CA, USA', network_type='drive')
nodes = ox.graph_to_gdfs(G, edges=False)

In [3]:
temp_cols = list(nodes.columns)
temp_cols[temp_cols.index('x')] = 'lng'; temp_cols[temp_cols.index('y')] = 'lat'
nodes.columns = temp_cols
nodes = nodes.sort_index()

In [4]:
# create mapping from original node number names to new node number names

node_name_map = {}; node_name_map_inv = {}
node_og = list(nodes.index)
num_nodes = len(node_og)
for i in range(num_nodes):
    node_name_map[i] = node_og[i]
    node_name_map_inv[node_og[i]] = i
nodes.index = range(nodes.shape[0])

In [6]:
df_police_stops = pd.read_csv('ca_san_francisco_2019_08_13.csv')
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lng'])]
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lat'])]

/anaconda3/envs/ox/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def haversine_np(long1, lat1, long2, lat2):
    """
    Calculate the great circle distance between an array of points to a single point
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    long1, lat1, long2, lat2 = map(np.radians, [long1, lat1, long2, lat2])

    dlon = long2 - long1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [8]:
from scipy.spatial import distance as dist
from time import time

coords = nodes[['lng', 'lat']].values
long1 = np.array(coords)[:, 0]
lat1 = np.array(coords)[:, 1]
batch_size = 500
police_stops = {}
m = df_police_stops.shape[0]
max_police_stops = 0
for i in range(0, m, batch_size):
    if i % 10000 == 0: print('%d/%d' % (i, m))
    if i + batch_size >= m:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:]
    else:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:i+batch_size]
    
    associated_nodes = np.array([haversine_np(long1, lat1, long2, lat2) for (long2, lat2) in stop_coords]).argmin(axis=1)
#     associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
    for idx in range(len(associated_nodes)):
        if associated_nodes[idx] in police_stops:
            new_police_stop = np.vstack([police_stops[associated_nodes[idx]], df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values])
        else:
            new_police_stop = np.reshape(df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values, [1, 2])
        
        if new_police_stop.shape[0] > max_police_stops: max_police_stops = new_police_stop.shape[0]
        police_stops[associated_nodes[idx]] = new_police_stop
#        police_stops[associated_nodes[idx]] = np.vstack([police_stops.get(associated_nodes[idx], ), df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values])#police_stops.get(associated_nodes[idx], []) + [df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values]
        

0/903373
10000/903373
20000/903373
30000/903373
40000/903373
50000/903373
60000/903373
70000/903373
80000/903373
90000/903373
100000/903373
110000/903373
120000/903373
130000/903373
140000/903373
150000/903373
160000/903373
170000/903373
180000/903373
190000/903373
200000/903373
210000/903373
220000/903373
230000/903373
240000/903373
250000/903373
260000/903373
270000/903373
280000/903373
290000/903373
300000/903373
310000/903373
320000/903373
330000/903373
340000/903373
350000/903373
360000/903373
370000/903373
380000/903373
390000/903373
400000/903373
410000/903373
420000/903373
430000/903373
440000/903373
450000/903373
460000/903373
470000/903373
480000/903373
490000/903373
500000/903373
510000/903373
520000/903373
530000/903373
540000/903373
550000/903373
560000/903373
570000/903373
580000/903373
590000/903373
600000/903373
610000/903373
620000/903373
630000/903373
640000/903373
650000/903373
660000/903373
670000/903373
680000/903373
690000/903373
700000/903373
710000/903373
720000

In [9]:
max_edges = 6
rewards = {}
worse_reward = -1000
for node in range(num_nodes):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
#     max_edges = max(max_edges, len(edges))
    # convert the edges from original names to new renamed, sorted is just to keep actions in some kind of order
    # for example the 1st action will always be the first chronological node based on naming
    edges = sorted([node_name_map_inv[edge] for edge in edges]) 
    # set reward of going to this edge equal to negative number of police stops at this node
    # if there are no police stops at the edge set reward to 0
    rewards[node] = [-(police_stops[edge].shape[0])/max_police_stops if edge in police_stops else 0 for edge in edges]
    rewards[node] = rewards[node] + (max_edges-len(rewards[node]))*[worse_reward]
    
# fill terrible reward for nodes that have less total actions, encodes impossible actions essentially
# for node in range(num_nodes):
#     curr_actions = rewards[node]
#     rewards[node] = curr_actions + [worse_reward]*(max_edges-len(curr_actions))

In [10]:
# np.unique(df_police_stops['subject_race'].values)
from threading import Thread


num_features = 2
def calc_reward(edge, sex, race):
#     if len(edge.shape) == 1: edge = edge.reshape(1, num_features)
    matching_sex = np.sum(edge[:, 0]==sex)
    matching_race = np.sum(edge[:, 1]==race)
    return -((.5 + .5*(matching_sex + matching_race)/num_features)*edge.shape[0])/max_police_stops

sexes = np.unique(df_police_stops['subject_sex'])
races = np.unique(df_police_stops['subject_race'].values)

def fill_state(node):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
    edges = sorted([node_name_map_inv[edge] for edge in edges])
    for sex in sexes:
        for race in races:
            state = (node, sex, race)
            police_rewards_plus[state] = [calc_reward(police_stops[edge], sex, race) if edge in police_stops else 0 for edge in edges]
            police_rewards_plus[state] = police_rewards_plus[state] + (max_edges-len(police_rewards_plus))*[worse_reward]
    
    print("Finished thread for node %d" % node)


police_rewards_plus = {}
nodes_list = list(range(num_nodes))

def check_threads(threads):
    count = 0
    for i in reversed(range(len(threads))):
        if not threads[i].isAlive():
            del threads[i]
            count += 1
    
    return count

threads = []
max_threads = 100

print("Starting %d threads" % max_threads)
for t in range(max_threads):
    next_node = nodes_list.pop(0)
    threads.append(Thread(target=fill_state, args=(next_node,)))

    threads[-1].start()
    
print("Finished starting %d threads" % max_threads)

while(len(nodes_list) > 0):
    count = check_threads(threads)
    if count > 0:
        for c in range(count):
            if len(nodes_list) == 0: break
            next_node = nodes_list.pop(0)
            
            threads.append(Thread(target=fill_state, args=(next_node,)))
            print("Starting thread for node %d" % next_node)
            threads[-1].start()
    else:
        continue
        
for t in threads:
    t.join()
# for node in range(num_nodes):
    
    

Starting 100 threads
Finished thread for node 0Finished thread for node 1

Finished thread for node 2
Finished thread for node 3
Finished thread for node 4
Finished thread for node 5
Finished thread for node 6
Finished thread for node 7
Finished thread for node 8
Finished thread for node 9
Finished thread for node 10
Finished thread for node 11
Finished thread for node 12
Finished thread for node 13
Finished thread for node 14
Finished thread for node 15
Finished thread for node 16
Finished thread for node 17
Finished thread for node 18
Finished thread for node 19
Finished thread for node 20
Finished thread for node 21Finished thread for node 22

Finished thread for node 23
Finished thread for node 24
Finished thread for node 25
Finished thread for node 26Finished thread for node 27

Finished thread for node 28
Finished thread for node 29Finished thread for node 30

Finished thread for node 31
Finished thread for node 32
Finished thread for node 33
Finished thread for node 34
Finished 

Finished thread for node 253Starting thread for node 254
Finished thread for node 254
Starting thread for node 255

Finished thread for node 255
Starting thread for node 256
Finished thread for node 256Starting thread for node 257

Finished thread for node 257
Starting thread for node 258
Finished thread for node 258
Starting thread for node 259
Finished thread for node 259
Starting thread for node 260
Finished thread for node 260Starting thread for node 261

Finished thread for node 261Starting thread for node 262

Finished thread for node 262
Starting thread for node 263
Finished thread for node 263
Starting thread for node 264
Finished thread for node 264Starting thread for node 265

Finished thread for node 265Starting thread for node 266
Finished thread for node 266
Starting thread for node 267

Finished thread for node 267
Starting thread for node 268
Finished thread for node 268Starting thread for node 269

Finished thread for node 269Starting thread for node 270

Finished threa

Finished thread for node 395Starting thread for node 396

Finished thread for node 396
Starting thread for node 397
Finished thread for node 397
Starting thread for node 398
Finished thread for node 398Starting thread for node 399

Finished thread for node 399Starting thread for node 400

Finished thread for node 400
Starting thread for node 401
Finished thread for node 401
Starting thread for node 402
Finished thread for node 402
Starting thread for node 403
Finished thread for node 403
Starting thread for node 404
Finished thread for node 404Starting thread for node 405
Finished thread for node 405

Starting thread for node 406
Finished thread for node 406
Starting thread for node 407
Finished thread for node 407
Starting thread for node 408
Finished thread for node 408
Starting thread for node 409
Finished thread for node 409Starting thread for node 410
Finished thread for node 410

Starting thread for node 411
Finished thread for node 411
Starting thread for node 412
Finished threa

Finished thread for node 607Starting thread for node 608

Finished thread for node 608
Starting thread for node 609
Finished thread for node 609Starting thread for node 610
Finished thread for node 610
Starting thread for node 611

Finished thread for node 611
Starting thread for node 612
Finished thread for node 612
Starting thread for node 613
Finished thread for node 613
Starting thread for node 614
Finished thread for node 614Starting thread for node 615

Finished thread for node 615Starting thread for node 616

Finished thread for node 616Starting thread for node 617

Finished thread for node 617
Starting thread for node 618
Finished thread for node 618Starting thread for node 619

Finished thread for node 619Starting thread for node 620
Finished thread for node 620

Starting thread for node 621
Finished thread for node 621
Starting thread for node 622
Finished thread for node 622Starting thread for node 623

Finished thread for node 623Starting thread for node 624

Finished threa

Starting thread for node 813
Finished thread for node 813Starting thread for node 814

Finished thread for node 814Starting thread for node 815

Finished thread for node 815
Starting thread for node 816
Finished thread for node 816
Starting thread for node 817
Finished thread for node 817Starting thread for node 818
Finished thread for node 818
Starting thread for node 819

Finished thread for node 819
Starting thread for node 820
Finished thread for node 820Starting thread for node 821
Finished thread for node 821
Starting thread for node 822

Finished thread for node 822
Starting thread for node 823
Finished thread for node 823
Starting thread for node 824
Finished thread for node 824
Starting thread for node 825
Finished thread for node 825
Starting thread for node 826
Finished thread for node 826
Starting thread for node 827
Finished thread for node 827Starting thread for node 828

Finished thread for node 828
Starting thread for node 829
Finished thread for node 829
Starting threa

...

Finished thread for node 8865Starting thread for node 8866

Finished thread for node 8866
Starting thread for node 8867
Finished thread for node 8867
Starting thread for node 8868
Finished thread for node 8868
Starting thread for node 8869
Finished thread for node 8869Starting thread for node 8870

Finished thread for node 8870
Starting thread for node 8871
Finished thread for node 8871
Starting thread for node 8872
Finished thread for node 8872
Starting thread for node 8873
Finished thread for node 8873
Starting thread for node 8874
Finished thread for node 8874Starting thread for node 8875

Finished thread for node 8875
Starting thread for node 8876
Finished thread for node 8876
Starting thread for node 8877
Finished thread for node 8877
Starting thread for node 8878
Finished thread for node 8878
Starting thread for node 8879
Finished thread for node 8879
Starting thread for node 8880
Finished thread for node 8880Starting thread for node 8881

Finished thread for node 8881
Starting t

Starting thread for node 9017
Finished thread for node 9017
Starting thread for node 9018
Finished thread for node 9018
Starting thread for node 9019
Finished thread for node 9019Starting thread for node 9020

Finished thread for node 9020
Starting thread for node 9021
Finished thread for node 9021
Starting thread for node 9022
Finished thread for node 9022
Starting thread for node 9023
Finished thread for node 9023Starting thread for node 9024

Finished thread for node 9024
Starting thread for node 9025
Finished thread for node 9025
Starting thread for node 9026
Finished thread for node 9026
Starting thread for node 9027
Finished thread for node 9027Starting thread for node 9028
Finished thread for node 9028
Starting thread for node 9029

Finished thread for node 9029
Starting thread for node 9030
Finished thread for node 9030
Starting thread for node 9031
Finished thread for node 9031Starting thread for node 9032
Finished thread for node 9032

Starting thread for node 9033
Finished t

Finished thread for node 9181
Starting thread for node 9182
Starting thread for node 9183Finished thread for node 9182

Finished thread for node 9183
Starting thread for node 9184
Finished thread for node 9184
Starting thread for node 9185
Finished thread for node 9185
Starting thread for node 9186
Finished thread for node 9186
Starting thread for node 9187
Finished thread for node 9187
Starting thread for node 9188
Finished thread for node 9188Starting thread for node 9189
Finished thread for node 9189

Starting thread for node 9190
Finished thread for node 9190
Starting thread for node 9191
Finished thread for node 9191
Starting thread for node 9192
Finished thread for node 9192
Starting thread for node 9193
Finished thread for node 9193
Starting thread for node 9194
Finished thread for node 9194
Starting thread for node 9195
Finished thread for node 9195Starting thread for node 9196
Finished thread for node 9196

Starting thread for node 9197
Finished thread for node 9197
Starting t

Finished thread for node 9359
Starting thread for node 9360
Finished thread for node 9360
Starting thread for node 9361
Finished thread for node 9361
Starting thread for node 9362
Finished thread for node 9362
Starting thread for node 9363
Finished thread for node 9363Starting thread for node 9364

Finished thread for node 9364Starting thread for node 9365

Finished thread for node 9365
Starting thread for node 9366
Finished thread for node 9366Starting thread for node 9367
Finished thread for node 9367
Starting thread for node 9368
Finished thread for node 9368
Starting thread for node 9369

Finished thread for node 9369
Starting thread for node 9370
Finished thread for node 9370
Starting thread for node 9371
Finished thread for node 9371
Starting thread for node 9372
Finished thread for node 9372Starting thread for node 9373

Finished thread for node 9373
Starting thread for node 9374
Finished thread for node 9374
Starting thread for node 9375
Finished thread for node 9375Starting th

Finished thread for node 9519Starting thread for node 9520

Finished thread for node 9520
Starting thread for node 9521
Finished thread for node 9521
Starting thread for node 9522
Finished thread for node 9522
Starting thread for node 9523
Finished thread for node 9523
Starting thread for node 9524
Finished thread for node 9524
Starting thread for node 9525
Finished thread for node 9525
Starting thread for node 9526
Finished thread for node 9526
Starting thread for node 9527
Finished thread for node 9527
Starting thread for node 9528
Finished thread for node 9528
Starting thread for node 9529
Finished thread for node 9529
Starting thread for node 9530
Finished thread for node 9530
Starting thread for node 9531
Finished thread for node 9531
Starting thread for node 9532
Finished thread for node 9532Starting thread for node 9533
Finished thread for node 9533

Starting thread for node 9534
Finished thread for node 9534
Starting thread for node 9535
Finished thread for node 9535Starting th

In [ ]:
# import pickle

# with open('nodes_with_police_stops.pickle', 'wb') as handle:
#     pickle.dump(police_stops, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('police_stop_rewards.pickle', 'wb') as handle:
#     pickle.dump(rewards, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('nodes_with_police_stops.pickle', 'rb') as handle:
#     police_stops = pickle.load(handle)

In [ ]:
# traffic_df = pd.read_csv('Traffic_Calming_Point_Features_092014.csv')

In [ ]:
# h = traffic_df['the_geom'].values
# buffer = 7
# points  = np.array([(float(i[i.find('(')+1:i[buffer:].find(' ')+buffer]), float(i[i[buffer:].find(' ')+1+buffer:i.find(')')])) for i in h])
# long, lat = points[:, 0], points[:, 1]
# del traffic_df['the_geom']
# traffic_df['lng'] = long
# traffic_df['lat']  = lat
# # [traffic_df['the_geom'].find('(')+1:traffic_df['the_geom'].find(' ')]

In [ ]:
# coords = nodes[['lng', 'lat']].values
# long1 = np.array(coords)[:, 0]
# lat1 = np.array(coords)[:, 1]
# batch_size = 100
# traffic_calmers = {}
# m = traffic_df.shape[0]
# for i in range(0, m, batch_size):
#     if i % 100 == 0: print('%d/%d' % (i, m))
#     if i + batch_size >= m:
#         stop_coords = traffic_df[['lng', 'lat']].values[i:]
#     else:
#         stop_coords = traffic_df[['lng', 'lat']].values[i:i+batch_size]
    
#     associated_nodes = np.array([haversine_np(long1, lat1, long2, lat2) for (long2, lat2) in stop_coords]).argmin(axis=1)
# #     associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
#     for idx in range(len(associated_nodes)):
#         traffic_calmers[associated_nodes[idx]] = traffic_calmers.get(associated_nodes[idx], []) + [traffic_df.iloc[i+idx].values]
        

In [ ]:
# VALUE ITERATION -- TOO LONG
import copy
import itertools

def calculate_transition(sPrime, state, reward):
    if reward == worse_reward:
        return 0.0
    else:
        numPossibleActions = sum(1 if r >= 0.0 else 0 for r in rewards[state]) # number of actual possible actions
        return 1.0 / numPossibleActions 

iteration = 1
discount = 0.95
U = pd.DataFrame(np.zeros((num_nodes, num_nodes)))
print("U", U)
# calculate all states
states = [s for s in itertools.product([i for i in range(num_nodes)], repeat=2)]
while(True):
    U_prev = U.copy()
    #algo 4.3 in sequential problems textbook
    for s in states:
        curr_node, end_node = s[0], s[1]
        values = []
        for r in rewards[curr_node]:
            if curr_node == end_node:
                val = 1.0 + discount*sum(calculate_transition(sPrime, curr_node, r)*U_prev.iloc[sPrime] 
                    for sPrime in states)
            else:
                val = r + discount*sum(calculate_transition(sPrime, curr_node, r)*U_prev.iloc[sPrime] 
                    for sPrime in states)
            values.append(val)  
        U.iloc[s] = max(values)    
    difference = np.square(U.sub(U_prev)).values.sum()
    print("difference: ", difference, "iteration: ", iteration)
    if (difference < 1.0) or (iteration >= 10):
        break
    iteration += 1
print(U)

In [ ]:
# SARSA

import copy
import itertools
import random

def choose_action(s):
    state_rewards = rewards[s[0]]
    poss_actions = []
    for r in state_rewards:
        if r >= 0.0: poss_actions.append(r)
    return poss_actions

# algo 5.3
iteration = 1
discount = 0.95
learningRate = 0.01
QValues = pd.DataFrame(np.zeros((num_nodes, num_nodes)))
states = [s for s in itertools.product([i for i in range(num_nodes)], repeat=2)]
while (True):
    prevQVals = QValues.copy()
    print("Iteration ", iteration)
    random_states = random.sample(states, 1000)
    for s in random_states:
        poss_actions = choose_action(s)
        # next action and next reward are the same
        if random.randint(1, 100) == 1:
            action = random.sample(poss_actions, 1)
        else:
            action = max(poss_actions)
        # how to calculate next state???
        QValues.iloc[state] +=
        
    difference = np.square(QValues.sub(prevQVals)).values.sum()
    print("Difference: ", difference)
    #Once the Q values have converged, break
    if difference < 1 * 10**-1 or iteration > 10:
        break
    iteration += 1